In [106]:
import sys
sys.path.insert(0, '..')
import pandas as pd
import numpy as np
import seaborn as sns
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from scipy.stats import chisquare, ks_2samp
from config import PREPARED_DATA_PATH, ORDERED_CATEGORIES, UNORDERED_CATEGORIES, CITY, POSITION

In [19]:
STRATIFY_COL = 'stratify_col'
RANDOM_STATE = 42


In [68]:
train = pd.read_pickle('../data/prepared/train.pkl')
target = pd.read_pickle('../data/prepared/target.pkl')
test = pd.read_pickle('../data/prepared/test.pkl')

In [9]:
def stratify_data(df_data, stratify_column_name, stratify_values, stratify_proportions, random_state=None):
    """Stratifies data according to the values and proportions passed in
    Args:
        df_data (DataFrame): source data
        stratify_column_name (str): The name of the single column in the dataframe that holds the data values that will be used to stratify the data
        stratify_values (list of str): A list of all of the potential values for stratifying e.g. "Male, Graduate", "Male, Undergraduate", "Female, Graduate", "Female, Undergraduate"
        stratify_proportions (list of float): A list of numbers representing the desired propotions for stratifying e.g. 0.4, 0.4, 0.2, 0.2, The list values must add up to 1 and must match the number of values in stratify_values
        random_state (int, optional): sets the random_state. Defaults to None.
    Returns:
        DataFrame: a new dataframe based on df_data that has the new proportions represnting the desired strategy for stratifying
    """
    df_stratified = pd.DataFrame(columns = df_data.columns) # Create an empty DataFrame with column names matching df_data

    pos = -1
    for i in range(len(stratify_values)): # iterate over the stratify values (e.g. "Male, Undergraduate" etc.)
        pos += 1
        if pos == len(stratify_values) - 1: 
            ratio_len = len(df_data) - len(df_stratified) # if this is the final iteration make sure we calculate the number of values for the last set such that the return data has the same number of rows as the source data
        else:
            ratio_len = int(len(df_data) * stratify_proportions[i]) # Calculate the number of rows to match the desired proportion

        df_filtered = df_data[df_data[stratify_column_name] == stratify_values[i]] # Filter the source data based on the currently selected stratify value
        df_temp = df_filtered.sample(replace=True, n=ratio_len, random_state=random_state) # Sample the filtered data using the calculated ratio
        
        df_stratified = pd.concat([df_stratified, df_temp]) # Add the sampled / stratified datasets together to produce the final result
        
    return df_stratified # Return the stratified, re-sampled data  

In [109]:
test[STRATIFY_COL] = test[[CITY]].apply(lambda  x:', '.join(map(str, x)), axis=1)
train[STRATIFY_COL] = train[[CITY]].apply(lambda  x:', '.join(map(str, x)), axis=1)

In [110]:
test_distr = (test.groupby(STRATIFY_COL)[STRATIFY_COL].count() / len(test))
stratify_values = test_distr.index
stratify_proportions = test_distr.tolist()

In [120]:
(test[CITY] == "Заместитель начальника службы корпоративных коммуникаций").sum()

0

In [121]:
(train[CITY] == "Заместитель начальника службы корпоративных коммуникаций").sum()

0

In [117]:
print(test[CITY].nunique())
print(train[CITY].nunique())

816
853


In [116]:
len(set(test[CITY]) ^ set(train[CITY]))

529

In [125]:
len(set(test[CITY]) & set(train[CITY]))

570

In [126]:
len(set(test[CITY]) | set(train[CITY]))

1099

In [127]:
set(test[CITY]) & set(train[CITY])

{'NA',
 'Абадзехская',
 'Абакан',
 'Абинск',
 'Агроном ',
 'Адлер',
 'Азов',
 'Актау',
 'Актобе',
 'Алапаевск',
 'Алатырь',
 'Алдан',
 'Алексин',
 'Алматы',
 'Алтай ',
 'Альметьевск',
 'Анадырь',
 'Анапа',
 'Ангарск',
 'Анжеро-Судженск',
 'Апатиты',
 'Апрелевка',
 'Арамиль',
 'Арзамас',
 'Армавир',
 'Арсеньев',
 'Артем',
 'Архангельск',
 'Асбест',
 'Астрахань',
 'Атырау',
 'Ачинск',
 'Багаевская',
 'Балаково',
 'Балашиха',
 'Балашов',
 'Бангладеш',
 'Барабинск',
 'Барда ',
 'Барнаул',
 'Батайск',
 'Батырево',
 'Безенчук',
 'Белая Калитва',
 'Белгород',
 'Белебей',
 'Белово',
 'Белокуриха',
 'Белорецк',
 'Белореченск',
 'Бердск',
 'Березники',
 'Березовский ',
 'Бийск',
 'Биробиджан',
 'Бирюч',
 'Благовещенск ',
 'Бологое',
 'Большой Камень',
 'Бор',
 'Борзя',
 'Борисоглебск',
 'Боровичи',
 'Братск',
 'Брянск',
 'Бугульма',
 'Бугуруслан',
 'Буденновск',
 'Бузулук',
 'Буй',
 'Быково ',
 'Валуйки',
 'Великие Луки',
 'Великий Новгород',
 'Великобритания',
 'Верхний Уфалей',
 'Верхняя Пышма

In [118]:
len(set(test[POSITION]) ^ set(train[POSITION]))

19102

In [122]:
len(set(test[POSITION]) | set(train[POSITION]))

22318

In [123]:
len(set(test[POSITION]) & set(train[POSITION]))

3216

In [119]:
set(test[POSITION]) ^ set(train[POSITION])

{'Ведущий (региональный) специалист по обороту алкоголя, алкогольному декларированию и ЕГАИС',
 'Заместитель начальника службы корпоративных коммуникаций',
 'Business Valuation Analyst',
 'Инженер-конструктор, ведущий инженер-конструктор',
 'Главный менеджер по развитию ОРККП',
 'Начальник отдела мотивации и бизнес-планирования',
 'Стилист Jo Malone',
 'Менеджер по работе с бизнес клиентами',
 'Менеджер строительных проектов',
 'Специалист СЭБ отдела по работе с просроченной задолженностью',
 'Специалист ВЭД',
 'Младший специалист отдела по подбору персонала (рекрутер)',
 'Механик - наладчик',
 'C&B manager / HR- аналитик/ Менеджер по компенсациям и льготам Службы HR-эффективности Округа',
 'Администратор локальной сети',
 'Старший менеджер отдела отдела сбыта',
 'Координатор по торговому оборудованию',
 'Монтажник шпд',
 'Менеджер по оформлению кредитов',
 'Ведущий инженер по закачке рабочего агента в пласт цеха подготовки перекачки нефти и газового конденсата',
 'Юрист суброгационной

In [112]:
stratified_train = stratify_data(
    df_data=train, stratify_column_name=STRATIFY_COL,
    stratify_values=stratify_values,
    stratify_proportions=stratify_proportions)

ValueError: a must be greater than 0 unless no samples are taken

In [87]:
for col in ORDERED_CATEGORIES + UNORDERED_CATEGORIES:
    f_obs = train[col].value_counts() / len(train)
    f_exp = test[col].value_counts() / len(test)
    common_idx = f_obs.index.intersection(f_exp.index)
    chisq, p_value = chisquare(f_obs=f_obs.loc[common_idx], f_exp=f_exp.loc[common_idx], ddof=1)
    print(col, 'chisq:', chisq, 'p_value:', p_value)

salary_rating chisq: 0.016009001456495645 p_value: 0.9994638573693502
team_rating chisq: 0.0007473694718391948 p_value: 0.9999945671906315
managment_rating chisq: 0.0032187373775179836 p_value: 0.9999514792174871
career_rating chisq: 0.01020243840127024 p_value: 0.9999870329455216
workplace_rating chisq: 0.0011174173900297376 p_value: 0.9999900689226666
rest_recovery_rating chisq: 0.00993272797025919 p_value: 0.9997375008704167
city chisq: inf p_value: 0.0
position chisq: inf p_value: 0.0


/home/as/miniconda3/lib/python3.9/site-packages/scipy/stats/stats.py:6707: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp


In [98]:
for col in ORDERED_CATEGORIES + UNORDERED_CATEGORIES:
    f_obs = train[col].value_counts() / len(train)
    f_exp = test[col].value_counts() / len(test)
    common_idx = f_obs.index.intersection(f_exp.index)
    chisq, p_value = ks_2samp(data1=f_obs.loc[common_idx], data2=f_exp.loc[common_idx])
    print(col, 'chisq:', chisq, 'p_value:', p_value)

salary_rating chisq: 0.4 p_value: 0.873015873015873
team_rating chisq: 0.2 p_value: 1.0
managment_rating chisq: 0.2 p_value: 1.0
career_rating chisq: 0.16666666666666666 p_value: 0.9999999999999998
workplace_rating chisq: 0.4 p_value: 0.873015873015873
rest_recovery_rating chisq: 0.2 p_value: 1.0
city chisq: 0.28844404003639673 p_value: 1.095385563738221e-40
position chisq: 0.48718523165158173 p_value: 0.0


In [96]:
for col in ORDERED_CATEGORIES + UNORDERED_CATEGORIES:
    f_obs = np.array([1,1,1,1,1,1,1])
    f_obs = f_obs / np.sum(f_obs)
    f_exp =  np.array([1,1,1,1,1,1,1])
    f_exp = f_exp / np.sum(f_exp)
    chisq, p_value = chisquare(f_obs=f_obs, f_exp=f_exp, ddof=1)
    print(col, 'chisq:', chisq, 'p_value:', p_value)

salary_rating chisq: 0.0 p_value: 1.0
team_rating chisq: 0.0 p_value: 1.0
managment_rating chisq: 0.0 p_value: 1.0
career_rating chisq: 0.0 p_value: 1.0
workplace_rating chisq: 0.0 p_value: 1.0
rest_recovery_rating chisq: 0.0 p_value: 1.0
city chisq: 0.0 p_value: 1.0
position chisq: 0.0 p_value: 1.0


In [82]:
1e-08

1e-08

In [73]:
a = train[ORDERED_CATEGORIES[0]].value_counts()
b = test[ORDERED_CATEGORIES[0]].value_counts()

In [76]:
a.index.intersection(b.index)

CategoricalIndex([5, 4, 3, 1, 2], categories=[1, 2, 3, 4, 5], ordered=True, dtype='category')

In [66]:
len(stratified_train)

50876

In [67]:
len(train)

50876